In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

W: Operation was interrupted before it could finish
^C


In [ ]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
!pip3 install pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 19.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=68726b7bde4955810913cccfb17f1b7ae42b38f9d7cc49020d9e2cb602b8adde
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [8]:
# Инициализация

from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col, max, sum, countDistinct

spark = SparkSession \
    .builder \
    .appName("Lab1_6407_Bykova") \
    .getOrCreate()

In [12]:
# Загрузка данных

import os
data_path = os.path.join(os.curdir, "data")
trips_path = os.path.join("trips.csv")
stations_path = os.path.join("stations.csv")

trips = spark.read \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("DateTimeFormat", 'M/d/y H:m') \
        .csv(trips_path)

stations = spark.read \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("DateTimeFormat", 'M/d/y H:m') \
        .csv(stations_path)

_______________________________________

### 1. Найти велосипед с максимальным временем пробега

In [13]:
from pyspark.sql.functions import col

max_distance = (trips.groupBy("bike_id").agg(sum(col("duration")).alias("duration"))).orderBy(col("duration").desc()).first()
print(max_distance)

Row(bike_id=535, duration=18611693)


### 2. Найти наибольшее геодезическое расстояние между станциями

In [14]:
print("Before crossJoin: ", stations.count())
joined_stations = stations.crossJoin(stations)
print("After crossJoin: ", joined_stations.count())
joined_stations.printSchema()

Before crossJoin:  70
After crossJoin:  4900
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)



In [16]:
# Вычисляем расстояние между всеми станциями и выбираем максимальное

from math import sin, cos, sqrt, atan2, radians

# Радиус Земли
R = 6373.0

def computeDistance(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

res = joined_stations.rdd.map(lambda x: computeDistance(x[2], x[3], x[9], x[10])).max()
print(res)

69.9428256877473


### 3. Найти путь велосипеда с максимальным временем пробега через станции

In [17]:
duration = trips.select("start_station_name", "end_station_name", "duration").orderBy(col("duration").desc()).first()
print(duration)

Row(start_station_name='South Van Ness at Market', end_station_name='2nd at Folsom', duration=17270400)


### 4. Найти количество велосипедов в системе

In [18]:
trips.select("bike_id").distinct().count()

700

### 5. Найти пользователей потративших на поездки более 3 часов

In [19]:
res =  (trips.groupBy("bike_id").sum("duration").withColumnRenamed("sum(duration)", "time")).filter("time>10800")
res.show()
res.count()

+-------+-------+
|bike_id|   time|
+-------+-------+
|    471|1718831|
|    496|1679568|
|    148| 332138|
|    463|1722796|
|    540|1752835|
|    392|1789476|
|    623|2037219|
|    243| 307458|
|    516|1896751|
|     31| 407907|
|    580|1034382|
|    137|1529200|
|    251|1282980|
|    451|1695574|
|     85|1214769|
|    458|1647080|
|     65| 216922|
|    588| 266415|
|    255| 396395|
|     53| 226389|
+-------+-------+
only showing top 20 rows



699